# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [2]:
# TODO: Import the necessary libs
# For example: 
import os
import json
import chromadb
from chromadb.utils import embedding_functions
from dotenv import load_dotenv
from tavily import TavilyClient
from typing import List
from pydantic import BaseModel, Field

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage
from lib.tooling import tool

In [3]:
# TODO: Load environment variables
load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
tavily_api_key = os.getenv('TAVILY_API_KEY') 
chroma_openai_api_key = os.getenv('CHROMA_OPENAI_API_KEY')

for key in [openai_api_key, tavily_api_key, chroma_openai_api_key]:
    assert key is not None

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [4]:
# TODO: Create retrieve_game tool
# It should use chroma client and collection you created
chroma_client = chromadb.PersistentClient(path="chromadb")
embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
    api_key=openai_api_key,
    model_name="text-embedding-3-small",
    api_base="https://openai.vocareum.com/v1"
)
collection = chroma_client.get_collection(
   "udaplay",
   embedding_function=embedding_fn
)

In [5]:
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game
class GameInfo(BaseModel):
    Platform: str = Field(description="Distribution platform of the game, e.g., Game Boy")
    Name: str = Field(description="Name of the game")
    YearOfRelease: int = Field(description="Year when the game was released for that platform")
    Description: str = Field(description="Additional details about the game")

@tool
def retrieve_game(query: str) -> List[GameInfo]:
      results = collection.query(
         query_texts=[query],
         n_results=3
      )['metadatas'][0]
      return list(map(lambda r: GameInfo(
            Platform=r['Platform'],
            Name=r['Name'],
            YearOfRelease=r['YearOfRelease'],
            Description=r['Description']
      ), results))

#### Evaluate Retrieval Tool

In [6]:
# TODO: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result
class EvaluationReport(BaseModel):
    useful: bool = Field(description="Whether the documents are useful to answer the question")
    description: str = Field(description="Description about the evaluation result")

@tool
def evaluate_retrieval(question: str, retrieved_docs: List[GameInfo]) -> EvaluationReport:
    llm = LLM(
        api_key=openai_api_key,
        temperature=0
    )
    prompt = [
        SystemMessage(
            content="Your task is to evaluate if the documents are enough to respond the query. "
            "Give a detailed explanation, so it's possible to take an action to accept it or not."
        ),
        UserMessage(
            content=f"Question: {question}\n\n"
            f"Retrieved Documents: {retrieved_docs}\n\n"
            "Based on the question and the retrieved documents, are the documents useful to answer the question? "
            "Answer with a boolean field 'useful' and a 'description' field explaining your reasoning."
        )
    ]

    llm_response = llm.invoke(prompt, response_format=EvaluationReport)

    return llm_response.content

#### Game Web Search Tool

In [7]:
# TODO: Create game_web_search tool
# Please use Tavily client to search the web
tavily_client = TavilyClient(api_key=tavily_api_key)

In [8]:
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 

@tool
def game_web_search(question: str) -> str:
    search_results = tavily_client.search(query=question, num_results=3)
    return search_results

### Agent

In [9]:
# TODO: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
class StructuredAnswer(BaseModel):
    question: str = Field(description="The original question asked by the user.")
    answer: str = Field(description="Your final answer to the user's question.")
    reasoning: str = Field(description="Your detailed reasoning process.")
    tools_used: List[str] = Field(description="List of tools you used to gather information.")
    citations: List[str] = Field(description="Citations from where you got the information.")

# Please refer to lib/agents.py for more details about Agent implementation
game_agent = Agent(
  openai_api_key=openai_api_key,
  model_name="gpt-4o-mini",
  tools=[retrieve_game, game_web_search],
  evaluation_tool=evaluate_retrieval,
  response_format=StructuredAnswer,
  instructions=(
    "You are a game industry expert agent."
  )
)

In [10]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?
queries = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X realeased for Playstation 5?",
    "What is the game that uses magenta in the landscape?"
]

answers = []

for q in queries:
    run_object = game_agent.invoke(
      query=q
    )
    answers.append(run_object)
    

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__
[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachin

In [11]:
for run_object in answers: 
    answer = json.loads(run_object.get_final_state()['messages'][-1].content)
    for key, value in answer.items():
        key_title = key.capitalize().replace("_", " ")
        print(f"➤ {key_title}:")
        if isinstance(value, list):
            for item in value:
                print(f"\t• {item}")
        else:
            print(f"\t{value}")
    print("\n")

➤ Question:
	When Pokémon Gold and Silver was released?
➤ Answer:
	Pokémon Gold and Silver were released in 1999.
➤ Reasoning:
	The question asks for the release date of Pokémon Gold and Silver. The retrieved information indicates that these games were released in 1999, which directly answers the question.
➤ Tools used:
	• retrieve_game
➤ Citations:
	• N/A


➤ Question:
	Which one was the first 3D platformer Mario game?
➤ Answer:
	The first 3D platformer Mario game is Super Mario 64, released in 1996.
➤ Reasoning:
	While there were earlier attempts at 3D platforming, such as Alpha Waves, Super Mario 64 is recognized as the first fully realized 3D platformer that set the standard for the genre. It introduced many core gameplay mechanics and a 3D world that worked effectively, making it a significant title in video game history.
➤ Tools used:
	• game_web_search
➤ Citations:
	• https://gamefaqs.gamespot.com/boards/198848-super-mario-64-disk-version/40777898
	• https://www.reddit.com/r/vid

### (Optional) Advanced

In [12]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes